# User Diversity

In [18]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

cu = db.users.find()
cs = db.shots.find()
ct = db.teams.find()
cutfs = db.user_team_first_shot.find()
cr = db.shots_ranking.find()

In [ ]:
def exist(user, lista):
    for u in lista:
        if u['username'] == user:
            return True
    return False   

def crea_users_in_teams():
    cutfs = db.user_team_first_shot.find()
    usernames_no_repeated = []
    for user in cutfs:
        if not exist(user['user'], usernames_no_repeated):
            usernames_no_repeated.append({'username': user['user'], 'team': user['team']})
            
    db.drop_collection('users_in_teams')
    for u in usernames_no_repeated:
        db.users_in_teams.insert_one(u)

In [25]:
skills_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/skills_frequences.txt", "r") as ins:
    for line in ins:
        skill = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'skill': skill, 'count': int(count)}
        skills_found.append(elem)

In [23]:
from decimal import *
getcontext().prec = 5

users_in_teams = db.users_in_teams.find()


# Numero di users che hanno skill
user_with_skills = []
for u in users_in_teams:
    active_user = db.skills.find({'user': u['username']})
    if active_user.count() > 0: 
        user_with_skills.append(active_user)

# clacola la prob che la skill 'skill' sia contenuto in uno user 
def skill_prob(skill):
    for s in skills_found:
        if s['skill'] == skill:
            return float(Decimal(s['count'])/Decimal(len(user_with_skills)))

# calcola l'informazione di una skill
def skill_info(skill):
    p_skill = skill_prob(skill)
    return (float(p_skill) * np.log2(float(p_skill)))*-1

# Calcolo della Diversity
def diversity(user):
    d = 0
    skills = db.skills.find({'user':user})[0]['skills']
    for s in skills:
        d = d + (float(skill_info(s)))
    return d   

/home/simo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()
